# NE555

Datasheet, monostable, reproduce figure 6-3:


- Vsup = +10V / 0V
- C = 1uF
- Ra = RL = 1kohm


Formula's :

- Ouput pulse duration : tw = 1.1*Ra*C


In [1]:
import json
import os
from pathlib import Path
import numpy as np

import matplotlib.pyplot as plt
import time

from Rigol.DS1104ZPLUS import capture_waveform, save_waveform_capture, load_waveform_capture

## Q1:

1. Generate a low impulsion on the generator.
2. Visualize onto the oscilloscope.

In [ ]:
from Rigol.RIGOL import RigolInstrument, detect_rigol_instruments

gen, osc = detect_rigol_instruments(verbose=1)

Ra = 10e3  # Ohm
RL = 10e3  # Ohm
C = 10e-6  # F   (uF)

# Output pulse duration
tw = 1.1 * Ra * C      # 110ms
expected_out_frequency = 1 / (2 * tw)  # Hz

# Datasheet "sequence ends only if TRIG is high for at least 10µs before the end of the timing interval" 
    # -> in pulse must be  short to triger the out
# trigger_pulse_width = 50e-6  # 50µs (short pulse to trigger)
# recovery_time = tw + 20e-3   # 130ms (> tw + 10µs safety margin)
# period = trigger_pulse_width + recovery_time  # ~130ms
# frequency = 1 / period  # ~7.7 Hz

# duty_cycle = (recovery_time / period) * 100

t_trig = 50e-6          # bref creux bas sur TRIG
T = tw + 20e-3 + t_trig # ~130 ms
frequency = 1 / T
duty_cycle = (T - t_trig) / T * 100.0  # ~99.96% haut

amplitude = 5.0
offset = 2.5

# 1. Generate the function
gen.generate_waveform(
    channel="CH1",
    waveform_type="PULS",
    frequency=frequency,
    amplitude=amplitude,
    offset=offset,
    duty_cycle=90,
)
time.sleep(1)

print(f"Duty cycle set to: 10%")
gen.write(f"PULS:DCYC 10")


# 2. oscilloscope

# Configuration explicite pour assurer la capture
# Trigger sur la sortie (CHAN2) car c'est elle qu'on mesure
osc.write(":TRIG:MODE EDGE")
time.sleep(1)
osc.write(":TRIG:EDG:SOUR CHAN2")
osc.write(":TRIG:EDG:SLOP NEG")    # Trigger sur flant descendant
osc.write(":TRIG:EDG:LEV 1.65")    # 1/3 des 5V threshold

# Echelles verticales estimées
osc.write(":CHAN1:SCAL 5.0")
time.sleep(1)
osc.write(":CHAN1:OFFS 0")
time.sleep(1)

# Base de temps: 20ms/div pour voir le front de montée
osc.write(":TIM:SCAL 20e-3")
time.sleep(1)
osc.write(":TIM:OFFS 60e-3")   # Offset 60ms to center pulse
time.sleep(1)


# 3. verif de la frequence
# query(f":MEAS:ITEM? VPP,CHAN{channel_in}"))


# Close
gen.close()
time.sleep(1)
osc.close()

Detecting...
Generator Detected.
Oscilloscope Detected.
Duty cycle set to: 10%


# Q2 :

1. Show the output of the Output Voltage (pin 3) and the Capacitor Voltage (pin 6 or 7).
2. Explain the obtained graph (both curves must superpose.)

In [ ]:


gen, osc = detect_rigol_instruments(verbose=1)
time.sleep(1)



# 1. Display Out (CH2) et CH3 (connecté a la capa)
osc.write(":CHAN1:DISP ON")
osc.write(":CHAN2:DISP ON")
osc.write(":CHAN3:DISP ON")



# Explications :
    # Le trigger enclenche le NE555 (flant qui descends)
    # Vout est immediatement haut
    # la capa (qui decide du timer°) se charge exponentiellement, et
    # quand elle arrive a 2/3 Vcc, le NE555 remet Vout a 0 et la capa se decharge
    # ducoup quand la capa atteint 1/3 Vcc, l'output est remit a 0.
    # et le cycle se repete





# Close
gen.close()
time.sleep(1)
osc.close()

Detecting...
Generator Detected.
Oscilloscope Detected.
